# Analyze Received Long Multi Tone Calibration WAV File
## By Terry Bondy, VA3TYB

In [1]:
printf(strftime ("Last updated: %A %e %B %Y %Z", localtime (time())))
%plot --format svg
format free
pkg load signal

Last updated: Monday 13 January 2020 UTC

In [2]:
inputFilename = "/mnt/lexar/SDRuno_20200107_005334Z_445900kHz_trimmed.wav"

inputFilename = /mnt/lexar/SDRuno_20200107_005334Z_445900kHz_trimmed.wav


In [3]:
audioinfo(inputFilename)

ans =

  scalar structure containing the fields:

    Filename = /mnt/lexar/SDRuno_20200107_005334Z_445900kHz_trimmed.wav
    CompressionMethod = 
    NumChannels = 2
    SampleRate = 62500
    TotalSamples = 1.40625e+07
    Duration = 225
    BitsPerSample = -1
    BitRate = -1
    Title = 
    Artist = 
    Comment = 



In [4]:
[SamplesIQRV, sampleRate] = audioread(inputFilename);
size(SamplesIQRV)
sampleRate

ans =

 1.40625e+07 2


sampleRate = 62500


In [5]:
SamplesRV=SamplesIQRV(:,1)+i*SamplesIQRV(:,2);
size(SamplesRV)

ans =

 1.40625e+07 1




In [6]:
# plot(abs(SamplesRV));

In [7]:
numPeriods = rows(SamplesRV)/sampleRate;
period = numPeriods;
TshortRV = [0:1/sampleRate:1](:);
TfullRV = [0:1/sampleRate:period](:);

In [8]:
RaisedCosineRV = (1 - cos(2*pi*TshortRV))/2;
size(RaisedCosineRV)

ans =

 62501 1




In [9]:
SigSec0RV = SamplesRV(1:rows(RaisedCosineRV)) .* RaisedCosineRV;
size(SigSec0RV)

ans =

 62501 1




In [10]:
# plot(abs(SigSec0RV));

In [11]:
FftSec0RV = abs(fftshift(fft(SigSec0RV, sampleRate*10 + 1)));
FreqSec0RV = [linspace(-sampleRate/2, sampleRate/2, rows(FftSec0RV))](:);

In [12]:
% plot(FreqSec0RV, FftSec0RV)
% grid on
% grid minor
% axis([100 800], "tic")

In [13]:
% plot(FreqSec0RV, FftSec0RV)
% grid on
% grid minor
% idx = find(FftSec0RV(:) == max(FftSec0RV))
% FftSec0RV(idx)
% FreqSec0RV(idx)
% axis([ans-4 ans+4], "tic")

In [14]:
% plot(FreqSec0RV, FftSec0RV)
% grid on
% grid minor
% axis([235-4 235+4], "tic")

In [15]:
find(FreqSec0RV(:) == 230)

ans = 314801


In [16]:
find(FreqSec0RV(:) == 245)

ans = 314951


In [17]:
idx = find(FftSec0RV(:) == max(FftSec0RV(314801:314951)))
FftSec0RV(idx)
FreqSec0RV(idx)

idx = 314855
ans = 141.949
ans = 235.4


In [18]:
FreqSec0RV(find(FftSec0RV(:) == max(FftSec0RV(314801:314951)))) - ...
FreqSec0RV(find(FftSec0RV(:) == max(FftSec0RV)))

ans = -521


In [19]:
721 - 521

ans = 200


In [37]:
fracSeconds = 1;
indexIncr = (rows(TshortRV)-1)/fracSeconds;
pilotFreq = 721;
Freqs = [];
# Use a most positive initial reference freq because probe freq starts low and increases
refFreq = sampleRate * 2;
for startIndex = 1:indexIncr:rows(SamplesRV)-rows(RaisedCosineRV)-1
  time = TfullRV(startIndex);
  # SigSec0RV = SamplesRV(1:rows(RaisedCosineRV)) .* RaisedCosineRV;
  SigLoopRV = SamplesRV(startIndex:startIndex+rows(RaisedCosineRV)-1) .* RaisedCosineRV;
  FftLoopRV = fftshift(fft(SigLoopRV, sampleRate*10 + 1));
  FftLoopAbsRV = abs(FftLoopRV);
  # max(FftLoopAbsRV)
  # FftLoopPolCV = cart2pol(real(FftLoopRV), imag(FftLoopRV)); # rows of [theta, r]
  # FftLoopPolCV(1,:)
  FreqLoopRV = linspace(-sampleRate/2, sampleRate/2, rows(FftLoopRV));
  freqMaxPeak = FreqLoopRV(find(FftLoopAbsRV(:) == max(FftLoopAbsRV)));
  [PeaksCV idxPeaksCV] = findpeaks(FftLoopAbsRV, "MinPeakHeight", 50);
  # size(PeaksCV)
  # size(idxPeaksCV)
  PeaksCV;
  if (rows(idxPeaksCV) == 1)
    idxPeaksCV = [idxPeaksCV; idxPeaksCV];
  endif
  FreqPeaksLoopRV = FreqLoopRV(idxPeaksCV);
  FreqDiff = abs(FreqPeaksLoopRV .- refFreq);
  minFreqDiff = min(FreqDiff);
  refFreqIdx = find(FreqDiff(:) == minFreqDiff);
  probeFreqIdx = rows(idxPeaksCV) + 1 - refFreqIdx;
  corrFreqLoopRV = FreqPeaksLoopRV .- FreqPeaksLoopRV(refFreqIdx) .+ pilotFreq;
  FftLoopRV(idxPeaksCV(probeFreqIdx))/FftLoopRV(idxPeaksCV(refFreqIdx));
  [phase, mag] = cart2pol(real(ans), imag(ans)); # rows of [theta, r]
  [corrFreqLoopRV(probeFreqIdx), phase*180/pi, 20*log10(mag)]
  
  # Compute the new reference frequency
  refFreq = FreqPeaksLoopRV(refFreqIdx);
endfor

ans =

 200 -47.1796 -9.15742


ans =

 200 -51.0027 -9.18264


ans =

 200.1 -72.0754 -9.37726


ans =

 207 9.8086 -8.84884


ans =

 207 6.28441 -8.78761


ans =

 207 3.24275 -9.00569


ans =

 213.9 85.5411 -8.4823


ans =

 213.9 81.9487 -8.45935


ans =

 214.1 42.9557 -8.63959


ans =

 222 -172.7 -8.03032


ans =

 222 -176.631 -8.06532


ans =

 222 -179.355 -8.26609


ans =

 230 -52.8348 -7.69709


ans =

 230 -56.4487 -7.66174


ans =

 230 -59.2745 -7.94056


ans =

 238 67.2102 -7.36767


ans =

 237.9 82.3044 -7.33449


ans =

 238 61.406 -7.57267


ans =

 246 -171.484 -6.9873


ans =

 245.9 -156.912 -7.01088


ans =

 246 -177.458 -7.30482


ans =

 255 11.9573 -6.69384


ans =

 255 8.16375 -6.74802


ans =

 255 5.90513 -6.9266


ans =

 264 -164.736 -6.34413


ans =

 264 -167.891 -6.38112


ans =

 264 -170.707 -6.61957


ans =

 273 19.258 -6.10446


ans =

 273 15.9484 -6.05728


ans =

 273 13.416 -6.30054


ans =

 283 -94.1883 -5.71374


ans =

 283 -97.5199